In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from main.utils import load_experiments
import os
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

### Pre-loading names

In [3]:
methods = [ 'random', 'entropy', 'bald', 'badge', 'max_logdet_S', 'similarity_kmeans']
datasets = ['mnist', 'dirty_mnist', 'repeated_mnist', 'fashion_mnist', 'imagenet']

# naming of methods
method_names = {
    'badge': 'BADGE',
    'random': 'Random',
    'entropy': 'Entropy',
    'bald': 'BALD',
    'max_logdet_S': 'Similarity LogDet',
    'empirical_covariance': 'Empirical Covariance',
    'similarity_kmeans': 'Similarity K-Means',
}


# naming of datasets
dataset_names = {
    'mnist': 'MNIST',
    'fashion_mnist': 'Fashion-MNIST',
    'dirty_mnist': 'Dirty-MNIST',
    'repeated_mnist': 'Repeated-MNIST',
    'imagenet': 'ImageNet',
}

n_seeds = 10

In [9]:
results = {}
for method in methods:
    results[method] = {}
    for dataset in datasets:
        experiment_ids = [f'{method}_{dataset}_20_to_200_B=20_{i + 1}' for i in range(n_seeds)]
        
        res = load_experiments(experiment_ids, base_dir='experiments_batch20')
        #res = load_experiments(experiment_ids, base_dir='experiments_dirty_mnist_10' if dataset == 'dirty_mnist' else 'experiments')
        accuracies = [[result[1]['test_accs'] for result in res]]
        results[method][dataset] = accuracies

# put all results in a single dataframe
data = []
for method in methods:
    for dataset in datasets:
        for result in results[method][dataset]:
            for j in range(n_seeds):
                data.append({
                    'method': method,
                    'dataset': dataset,
                    'seed': j,
                    'accuracy': result[j][-1],
                })

df = pd.DataFrame(data)

## Friedman + Nemenyi

In [10]:
# Reshape DataFrame for the Friedman Test
pivot_df = df.pivot_table(index=['dataset', 'seed'], columns='method', values='accuracy')

# Perform Friedman Test
friedman_stat, p_value = friedmanchisquare(*[pivot_df[method] for method in pivot_df.columns])

print(f"Friedman Test Statistic: {friedman_stat}, p-value: {p_value}")

# Perform Nemenyi Post-hoc Test
nemenyi_results = sp.posthoc_nemenyi_friedman(pivot_df.values)

# Format Nemenyi test results
method_names = pivot_df.columns
nemenyi_df = pd.DataFrame(nemenyi_results)
nemenyi_df.columns = method_names
nemenyi_df.index = method_names


# Display styled DataFrame
nemenyi_df.to_csv('nemenyi_results_20_excluding.csv')

Friedman Test Statistic: 94.40000000000009, p-value: 7.985542734556428e-19
